In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import os
print("Tensorflow version " + tf.__version__)

resolver = tf.distribute.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)

# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

strategy = tf.distribute.TPUStrategy(resolver)

Tensorflow version 2.8.0
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.67.45.82:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.67.45.82:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
import numpy as np
import keras
import time
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, EarlyStopping,Callback
from keras.layers import Dense,Reshape,Input,BatchNormalization,Dropout,concatenate,Flatten,Lambda,Resizing,CuDNNLSTM,concatenate,Layer,LSTM,Conv2D,MaxPool2D
from keras import  Model,layers
from tensorflow.keras.applications import VGG16
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [ ]:
def find_removable(n,batch_size=64):
      return n%batch_size

In [ ]:
base_path = '/content/drive/MyDrive/Colab Notebooks/OAD/OAD/Vincent/NEW/oad_fusion_2img/40/'
# base_path = '/content/drive/MyDrive/Colab Notebooks/NTU/Vincent/ntu_3cam/'
input_shape= (50,40,3)

train_x = np.load(base_path+'train_x.npy')
test_x = np.load(base_path+'test_x.npy')
train_y = np.load(base_path+'train_y.npy')
test_y = np.load(base_path+'test_y.npy')
val_x = test_x
val_y = test_y
np.random.seed(11)
np.random.shuffle(val_x)
np.random.seed(11)
np.random.shuffle(val_y)
val_x = val_x[:1000]
val_y = val_y[:1000]

batch_size=32

n = find_removable(len(train_x),batch_size)
train_x = train_x[:-n]
train_y = train_y[:-n]

print('lost',n)

train = (train_x,train_y)
test = (test_x,test_y)

lost 31


In [ ]:
with strategy.scope(): # creating the model in the TPUStrategy scope means we will train the model on the TPU
  inp=Input(np.asarray(input_shape))
  output = Conv2D(16,padding='same',activation='relu',kernel_size=(3,3))(inp)
  output = MaxPool2D(pool_size=(2,2),strides=2)(output)
  output = Conv2D(32,padding='same',activation='relu',kernel_size=(3,3))(output)
  output = MaxPool2D(pool_size=(2,2),strides=2)(output)
  output = Conv2D(64,padding='same',activation='relu',kernel_size=(2,2))(output)
  output = Conv2D(64,padding='same',activation='relu',kernel_size=(2,2))(output)
  output = MaxPool2D(pool_size=(2,2),strides=2)(output)
  output = Conv2D(128,padding='same',activation='relu',kernel_size=(2,2))(output)
  output = Conv2D(128,padding='same',activation='relu',kernel_size=(2,2))(output)
  output = Conv2D(128,padding='same',activation='relu',kernel_size=(2,2))(output)
  output = MaxPool2D(pool_size=(2,2),strides=2)(output)
  output = Conv2D(256,padding='same',activation='relu',kernel_size=(2,2))(output)
  output = Conv2D(256,padding='same',activation='relu',kernel_size=(2,2))(output)
  output = Conv2D(256,padding='same',activation='relu',kernel_size=(2,2))(output)
  output = MaxPool2D(pool_size=(2,2),strides=2)(output)
  output = Flatten()(output)
  output = Dense(256,activation='relu',kernel_regularizer='l1')(output)
  output = BatchNormalization()(output)
  output = Dense(10, activation='softmax',kernel_regularizer='l1')(output)
  model = Model(inp, output)
  model.summary()
  optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
  model.compile(loss=keras.losses.categorical_crossentropy,
                optimizer=optimizer,
                metrics=['accuracy'])

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 50, 40, 3)]       0         
                                                                 
 conv2d_68 (Conv2D)          (None, 50, 40, 16)        448       
                                                                 
 max_pooling2d_37 (MaxPoolin  (None, 25, 20, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_69 (Conv2D)          (None, 25, 20, 32)        4640      
                                                                 
 max_pooling2d_38 (MaxPoolin  (None, 12, 10, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_70 (Conv2D)          (None, 12, 10, 64)        8256

In [ ]:
filepath ="oad.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=0, save_best_only=True, mode='max')
callbacks = [checkpoint,EarlyStopping(monitor='val_accuracy', patience=100,)]
debut = time.time()
history=model.fit(train[0], train[1],
              epochs=300,
              batch_size=batch_size,
              validation_data=(val_x,val_y),
              callbacks=callbacks,
              verbose=1)
fin = time.time()
print("Temps total : " + str(fin - debut) + "ms")
model = keras.models.load_model('oad.hdf5')
keras.models.save_model(model, base_path+'_perso_64.h5')
print(model.evaluate(test[0],test[1]))

Epoch 1/300
372/372 [==============================] - 13s 26ms/step - loss: 37.0519 - accuracy: 0.7047 - val_loss: 35.9678 - val_accuracy: 0.6240
Epoch 2/300
372/372 [==============================] - 6s 16ms/step - loss: 34.1692 - accuracy: 0.8510 - val_loss: 33.2653 - val_accuracy: 0.7470
Epoch 3/300
372/372 [==============================] - 6s 16ms/step - loss: 31.6938 - accuracy: 0.8905 - val_loss: 30.9112 - val_accuracy: 0.7630
Epoch 4/300
372/372 [==============================] - 6s 16ms/step - loss: 29.3503 - accuracy: 0.9151 - val_loss: 28.5492 - val_accuracy: 0.8200
Epoch 5/300
372/372 [==============================] - 6s 16ms/step - loss: 27.1233 - accuracy: 0.9328 - val_loss: 26.3261 - val_accuracy: 0.8350
Epoch 6/300
372/372 [==============================] - 6s 17ms/step - loss: 24.9925 - accuracy: 0.9455 - val_loss: 24.2681 - val_accuracy: 0.8470
Epoch 7/300
372/372 [==============================] - 6s 16ms/step - loss: 22.9718 - accuracy: 0.9536 - val_loss: 22.2103 